# Cayley–Hamilton Theorem (CHT)

**Every square matrix in the real or complex field satisfies its own characteristic (eigenvalue) equation.**

# Controllability and Reachability

If and only if a system is controllable ie the controllability matrix $C(A, B)$ has full-row rank then any state is reachable.

Let us consider an example for control system eqaution: $Y = Ax+Bu$

Then $x$ is steerable to any vector in $R^n$ with some control input $u$.
If $y = eAt$ then by CHT it can be written as the sum of finite series of powers of $A$ upto $A^{n-1}$ with time varying coefficients.

#  Reachability

If a vector is reachable then an n ways of input control vector can be used to reach that vector.

# Using Cayley Hamilton Theorem

The vector can be represented as linear combination of finite scalar convolutions set of terms or integrals and controllable matrix.

Controllability is equivalent to reachability and vice versa.

If system is controllable then there are infinite control $U$ to reach the vector.

As $U$ is continuously varying function there are multiple degrees of freedom than $n \cdot q$ to reach the vector.

CHT helps to write convolution integral in terms of controllability matrix.

![](./images/ReachabilityControlability.png)

# Control a system in Matlab (Python)

## Inverted Pendulum

In Matlab / Python after developing the inverted pendulum equations using Jacobian Matrix and plugin the fixed points and linearized pendulum up and down condition and then check for controllability. One of eigen value of $A$ is unstable then the pendulum is moving To check controllability `Ctrb(A,B)4*4` if its rank is 4 then  system is controllable, else not. Further study can be made on stabilizing the system by measuring its parameters.

![](./images/InvertedPendulumMatlab.png)

State of system
- Position of cart: $x$, angle of pendulum arm: $\Theta$
- 2 degrees of freedom system
- Its nonlinearity can be linearized
- Fixed points: Pendulum up or down and no velocities

## Python (Matlab)
Matlab code here makes only sense, if you have a Matlab kernel (incl. toolboxes) or an Octave equivalent. The original Matlab code which was listed in `Python` cells can be found at [GitHub](https://github.com/bertozzijr/Control_Bootcamp_S_Brunton).

- [Inverted pendulum - Wikipedia](https://en.wikipedia.org/wiki/Inverted_pendulum)
- [Control Tutorials for MATLAB and Simulink - Inverted Pendulum: System Modeling](https://ctms.engin.umich.edu/CTMS/index.php?example=InvertedPendulum&section=SystemModeling)
- [numpy.linalg.eigvals - NumPy Manual](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eigvals.html#numpy-linalg-eigvals)
- [scipy.integrate.ode - SciPy Manual](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.ode.html)

- [cartpend.m](https://github.com/bertozzijr/Control_Bootcamp_S_Brunton/blob/master/cartpend.m)

In [1]:
def cartpend(y, m, M, L, g, d, u):
    # Nonlinear ODE (y: State vector (x in talk); m: Mass of pendulum head, M: Cart mass; L: Length of pendulum 
    # g: Gravitational acceleration, d: Damping factor, u: Controll input vector)
    Sy = np.sin(y[2])
    Cy = np.cos(y[2])
    D = m * L * L * (M + m * (1 - Cy**2))
    dy = np.zeros((4, 1))
    dy[0, 0] = y[1]
    dy[1, 0] = (1 / D) * (-m**2 * L**2 * g * Cy * Sy + m * L**2 * (m * L * y[3]**2 * Sy - d * y[1])) + m * L * L * (1 / D) * u
    dy[2, 0] = y[3]
    dy[3, 0] = (1 / D) * ((m + M) * m * g * L * Sy - m * L * Cy * (m * L * y[3]**2 * Sy - d * y[1])) - m * L * Cy * (1 / D) * u + 0.01 * np.random.randn()
    return dy

- [poleplace_cartpend.m](https://github.com/bertozzijr/Control_Bootcamp_S_Brunton/blob/master/poleplace_cartpend.m)

In [22]:
import numpy as np

m = 1 # Mass of pendulum head
M = 5 # Cart mass
L = 2 # Length of pendulum
g = -10 # Gravitational acceleration
d = 1 # Damping factor due to cart frictions (pendulum friction assumed to be negligible)
s = 1 # pendulum up (s=1) / down (s=-1) switch

A = np.array([[0, 1, 0, 0], 
              [0, -d/M, -m*g/M, 0], 
              [0, 0, 0, 1], 
              [0, -s*d/(M*L), -s*(m+M)*g/(M*L), 0]])
B = np.array([[0], [1/M], [0], [s*1/(M*L)]])

A, B

(array([[ 0. ,  1. ,  0. ,  0. ],
        [ 0. , -0.2,  2. ,  0. ],
        [ 0. ,  0. ,  0. ,  1. ],
        [ 0. , -0.1,  6. ,  0. ]]),
 array([[0. ],
        [0.2],
        [0. ],
        [0.1]]))

In [21]:
np.linalg.eigvals(A) # 0, -2.4674, -0.1665, 2.4339

array([ 0.        , -2.46742895, -0.16651192,  2.43394087])

In [29]:
import control as ct

# ans =    0    0.2000   -0.0400    0.2080
#     0.2000   -0.0400    0.2080   -0.0816
#          0    0.1000   -0.0200    0.6040
#     0.1000   -0.0200    0.6040   -0.1408
C = ct.ctrb(A, B) # Controlability matrix

In [28]:
np.linalg.matrix_rank(C) # Is it controllable?

4

- [drawcartpend.m](https://github.com/bertozzijr/Control_Bootcamp_S_Brunton/blob/master/drawcartpend.m)

In [25]:
# def drawcartpend(y, m, M, L):
#     x = y(1)
#     th = y(3)
#     W = 1 * sqrt(M / 5)
#     H = 0.5 * sqrt(M / 5)
#     wr = 0.2
#     mr = 0.3 * sqrt(m)
#     y = (wr / 2) + (H / 2)
#     w1x = x - ((0.9 * W) / 2)
#     w1y = 0
#     w2x = (x + ((0.9 * W) / 2)) - wr
#     w2y = 0
#     px = x + (M[L] @ sin(th))
#     py = y - (M[L] @ cos(th))
#     plot(M[[-10, 10]], M[[0, 0]], "k", "LineWidth", 2)
#     hold("on")
#     rectangle("Position", M[[x - (W / 2), y - (H / 2), W, H]], "Curvature", 0.1, "FaceColor", M[[1, 0.1, 0.1]],)
#     rectangle("Position", M[[w1x, w1y, wr, wr]], "Curvature", 1, "FaceColor", M[[0, 0, 0]])
#     rectangle("Position", M[[w2x, w2y, wr, wr]], "Curvature", 1, "FaceColor", M[[0, 0, 0]])
#     plot(M[[x, px]], M[[y, py]], "k", "LineWidth", 2)
#     rectangle("Position", M[[px - (mr / 2), py - (mr / 2), mr, mr]], "Curvature", 1, "FaceColor", M[[0.1, 0.1, 1]],)
#     xlim(M[[-5, 5]])
#     ylim(M[[-2, 2.5]])
#     set(gcf, "Position", M[[100, 550, 1000, 400]])
#     drawnow
#     hold("off")

- [sim_cartpend.m](https://github.com/bertozzijr/Control_Bootcamp_S_Brunton/blob/master/sim_cartpend.m)

In [15]:
# m = 1
# M = 5
# L = 2
# g = -10
# d = 1
# tspan = M[0:0.1:10]
# y0 = M[0, 0, np.pi, 0.5,]
# for k in M[1 : length(t)]:
#     drawcartpend_bw(y[I[k, :]], m, M, L)

![](./images/InvertedPendulum.png)

- [poleplace_cartpend.m](https://github.com/bertozzijr/Control_Bootcamp_S_Brunton/blob/master/poleplace_cartpend.m)

In [16]:
# %%  Pole placement

# % p is a vector of desired eigenvalues
# % p = [-.01; -.02; -.03; -.04]; % not enough
# p = [-.3; -.4; -.5; -.6];  % just barely
# p = [-1; -1.1; -1.2; -1.3]; % good
# p = [-2; -2.1; -2.2; -2.3]; % aggressive
# p = [-3; -3.1; -3.2; -3.3]; % aggressive
# % p = [-3.5; -3.6; -3.7; -3.8]; % breaks
# K = place(A,B,p);
# % K = lqr(A,B,Q,R);

# tspan = 0:.001:10;
# if(s==-1)
#     y0 = [0; 0; 0; 0];
#     %[t,y] = ode45(@(t,y)cartpend(y,m,M,L,g,d,-K*(y-[4; 0; 0; 0])),tspan,y0);
# elseif(s==1)
#     y0 = [-3; 0; pi+.1; 0];
# %     [t,y] = ode45(@(t,y)cartpend(y,m,M,L,g,d,-K*(y-[1; 0; pi; 0])),tspan,y0);
#    % [t,y] = ode45(@(t,y)cartpend(y,m,M,L,g,d,-K*(y-[1; 0; pi; 0])),tspan,y0);
# else 
# end

# for k=1:100:length(t)
#     drawcartpend_bw(y(k,:),m,M,L);
# end